# The Deep Learning Book (Simplified)
## Part I - Applied Math and Machine Learning basics
*This is a series of blog posts on the [Deep Learning book](http://deeplearningbook.org) where I am attempting to provide a summary of each chapter highlighting the concepts that I found to be most important so that other people can use it as a starting point for reading the chapters, while including the code for reproducing some of the results. Please refer [this](http://www.deeplearningbook.org/contents/notation.html) for more clarity on notation.*

## Chapter 4: Numerical Computation

Since you are here, there's a high probability that you must have heard of **Gradient Descent**. It is that part of a Deep Learning pipeline which leads to the model being *trained*. This chapter outlines the various kinds of numerical computations generally utilized by Machine Learning algorithms and also describes various optimization algorithms (e.g. Gradient Descent, Newton's method), which are those class of algorithms that update the estimates of the solution iteratively, rather than solving it analytically to provide a closed-form solution.

The sections present in this chapter are listed below: <br>

**1. Overflow and Underflow?** <br>
**2. Poor Conditioning** <br>
**3. Gradient-Based Optimization** <br>
**4. Constrained Optimization** <br>

### 1. Overflow and Underflow

There is a fundamental problem with representing infinitely many real numbers on a digital computer with a finite number of bit patterns, which is: it leads to rounding errors. Such rounding errors compound over certain operations and cause many theoretically correct algorithms to fail in practise. There are primarily two damaging forms of rounding errors:

- **Underflow**: Underflow occurs when numbers near to zero are rounded down to zero. <br> 
The behaviour of certain functions like $\frac{1}{x}$ , $log$, etc. can change dramatically due to this.

- **Overflow**: Overflow occurs when a large number is approximated as $\infty$ (or $-\infty$).

*Example* - Softmax
![softmax](images/softmax.png)

Assume every $x_i$ is equal to some $c$. <br> 

**Problems**:
- $c$ is very negative: This leads to underflow when computing $exp(c)$ and thus $0$ in the denominator.
- $c$ is very positive: This leads to overflow when computing $exp(c)$.

**Solution**: 

Instead of computing $softmax(\mathbf{x})$, we compute $softmax(\mathbf{z})$, where $\mathbf{z} = \mathbf{x} - \max_i x_i$. It can be proven that the value doesn't change after subtracting the same value from each of the elements. Now, the maximum value in $\mathbf{z}$ is $0$, thus preventing overflow. Also, this ensures that atleast one element in the denominator is $1$, preventing underflow.

*Food for thought*: This still doesn't prevent underflow in the numerator. Think of the case when the output from the softmax function is passed as input to another function, e.g., $log$.

### 2. Poor Conditioning

Conditioning measures how rapidly the output of a function changes with small changes in the input. Large conditioning means poor conditioning as rounding errors can lead to large changes in output.
For e.g., let's observe: $ f(x) = A^{-1}x$. Given that $A \in \mathbb{R}^{n \hspace{.1cm} \text{x} \hspace{.1cm} n}$ has an eigen value decomposition, its **condition number** is given by:

![condition number](images/condition_number.png)

which is equal to the ratio of the largest and the smallest eigen values. Having a large condition number signifies that matrix inversion is highly sensitive to errors in the input.

### 3. Gradient Based Optimization

Most optimization problems are phrased in terms of minimizing $f(x)$. Maximization can be achieved via a minimization algorithm by minimizing $-f(x)$.

The **derivative** of a function $f$, denoted as $f'(x)$, specifies how a small change in input reflects as a change in output: $f(x + \epsilon) \approx f(x) + \epsilon * f'(x)$. The derivative is useful for minimizing a function because it tells us how to change $x$ in order to make a small improvement in $y$. E.g. for a small enough $\epsilon$, $f\Big(x - \epsilon\, sign\big(f'(x)\big) \Big)$ will be smaller than $f(x)$. This technique is called **gradient descent**.

Points where $f'(x)=0$ are called **critical** or **stationary** points. Types of critical points:

![critical points](images/critical_points.png)

For functions with multiple inputs, **partial derivative** $\frac{\delta}{\delta x_i}f(x)$ measures how $f$ changes as only the variable $x_i$ changes at point $x$. The **gradient** of $f$ is a vector containing all partial derivatives denoted $\nabla_x\, f(x)$. The **directional derivative** in a direction ***u*** (unit vector) is the slope of $f$ in the direction *u*

i.e. the directional derivative is the value of $\frac{\delta}{\delta \alpha}f(x+\alpha*u)$ evaluated as $\alpha=0$.

Using the chain rule:

$\frac{\delta}{\delta \alpha}f(x+\alpha*u) = \big(\frac{\delta}{\delta \alpha}(x+\alpha*u)\big)^T\frac{\delta}{\delta(x+\alpha*u)}f(x+\alpha*u)$

as $\alpha$ tends to 0 the expression reduces to $u^T\nabla_x\, f(x)$. To minimize $f$ we need to find the direction *u* in which $f$ decreases the fastest i.e.:

![partial derivative](images/partial_derivative.png)

Ignoring terms not relating to *u* we see that function *f* is decreased most when $cos\theta = -1$ i.e. we move in the direction opposite to the gradient. This is the method of **steepest descent** or **gradient descent**. Steepest descent proposes the new point: $x' = x - \epsilon \nabla_x\, f(x)$ where $\epsilon$ is the **learning rate**. $\epsilon$ can be a small constant or can be solved analytically to make the gradient vanish. Another approach is to try different values of $\epsilon$ and choose the value that causes the most decrease (**line search**).

The general concept of repeatedly making a small move in the locally best direction can be generalized to discrete spaces (**hill climbing**).

#### 3.1 Jacobian and Hessian Matrices

If we have a function $f: \mathbb{R}^m \rightarrow \mathbb{R}^n$, then the **Jacobian** matrix $J \in \mathbb{R}^{n\, \times\, m}$ of $f$ is defined such that $J_{i,j} = \frac{\delta}{\delta x_j}f_i(x)$

The **second derivative** tells us how the first derivative changes with small changes in input. It is a measurement of **curvature**. The **Hessian** matrix **H**(f)(**x**) is defined such that

$H(f)(x)_{i,j} = \frac{\delta^2}{\delta x_i \delta x_j}f(x)$

Hessian is the Jacobian of the gradient.

### 4. Constrained Optimization

It might be the case that although we want to maximize (or minimize) $f(x)$, but aren't allowed to use all possible values of $x$, say $x \in \mathbb{S}$, for some set $\mathbb{S}$. This now becomes a problem of **Constrained Optimization**. The points $\mathbf{x}$ in $S$ are called **feasible points**. 

An example of such a constraint can be the L2-norm constraint, e.g. $|| \hspace{.1cm} x \hspace{.1cm}||^2 < 1$. This is useful as we often want the values for our weights to be small (i.e. close to $0$).

*Approach*: Design a separate, unconstrained optimization problem, whose solution can be converted to the original constrained optimization problem. E.g. in the above described constrained optimization problem, we could instead minimize:
$$g(\theta) = f([\cos\theta, \sin\theta]^T)$$

with respect to $\theta$ and return ($\cos\theta, \sin\theta$).


General solution: **Karush–Kuhn–Tucker(KKT)** approach which introduces a **generalized Lagrangian**.

Approach: 

We use $m$ functions $g^{(i)}(x)$ and $n$ functions $h^{(j)}(x)$ to describe $\mathbb{S}$,  such that any element $x \in \mathbb{S}$ satisfies: 
$$g^{(i)}(x) = 0 \hspace{.1cm} \text{and} \hspace{.1cm} h^{(j)}(x) \leq 0 \hspace{.1cm} \forall \hspace{.1cm} i, j$$

There are two constraints specified here. I'll explain them with an example. Let's take $g(x)$ as $x - 2$ and $h(x)$ as $x-3$. <br>
Then for $x = 2$, we have the following:

- **Equality constraints**: $g^{(i)}(x) = 0$. Here, $g(2) = 0$. Hence, $x = 2$ satisfies the equality constraints.
- **Inequality constraints**: $h^{(i)}(x) \leq 0$. Here, $h(2) = -1 < 0$. Hence, $x = 2$ satisfies the inequality constraints.

Note that for $x = 3$, $h(x)$ is an equality constraint that it satisfies whereas $g(x)$ is neither.

New paramaters (called KKT multipliers): $\lambda_i$, $\alpha_j$ for each constraint.  <br>
Generalized Lagrangian:


![lagrangian](images/Lagrangian.png)

Now, let: $Y =\max\limits_{\alpha} \max\limits_{\lambda} L(x, \lambda, \alpha)$
Then, $\min\limits_x(f(x)) = \min\limits_x(Y)$

This is because, if the constraints are satisfied, $Y = f(x)$ and if it isn't, $Y = \infty$. This ensures that only feasible points are optimal. For finding the maximum of f(x), we can use the same generalized Lagrangian applied on $-f(x)$. 

The inequality constraints need to be observed more closely. Suppose the optimal point comes out to be $x^*$. If $h^{(i)}(x^*) = 0$, then the constraint is said to be **active**. However, if the constraint is inactive, i.e. $h^{(i)}(x^*) < 0$, then even if we remove the constraint, $x^*$ continues to be a local solution. Also, by definition, an inactive $h^{(i)}$ is negative and hence $\max\limits_{\alpha} \max\limits_{\lambda} L(x, \lambda, \alpha) \Rightarrow \alpha_i = 0$. Thus, either $\alpha_i = 0$ or $h^{(i)}(x^*) = 0$ (in the case of active constraint). Hence, $\mathbf{\alpha} \odot h{(x)} = 0$.

Intuition: 

The relation of the optimal point can satisfy only of these two conditions:

- The point is at the boundary of the constraint (i.e. active), then the corresponding KKT multiplier should be used.

- The constraint has no influence in the evaluation of the point and hence, the corresponding KKT multiplier is zeroed out.

The optimal points satisfy the following KKT conditions, which are necessary but not always sufficient:

![kkt](images/kkt.png)